### 前準備

In [1]:
# データセットのダウンロード
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
! unzip NewsAggregatorDataset.zip
# 読込時のエラー回避のためダブルクォーテーションをシングルクォーテーションに置換
!sed -e 's/"/'\''/g' ./newsCorpora.csv > ./newsCorpora_re.csv
from gensim.models import KeyedVectors
# 学習済み単語ベクトルのロード
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Colab Notebooks/nlp100/chapter8/GoogleNews-vectors-negative300.bin.gz', binary=True)

--2022-03-04 11:57:54--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  13.3MB/s    in 2.1s    

2022-03-04 11:57:57 (13.3 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203/29224203]

Archive:  NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [2]:
# ============
# 70. 単語ベクトルの和による特徴量
# ============
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv('/content/newsCorpora.csv', sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

# df.locは単独および複数の要素の値を選択、取得、変更が可能
# isinはbool型を返す
df1 = df.loc[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']), ['TITLE', 'CATEGORY']]

# データの分割 stratifyを設定することで訓練データとテストデータの指定した中身の割合を同じにすることができる
train, temp = train_test_split(df1, test_size=0.2, shuffle=True, random_state=0, stratify=df1['CATEGORY'])
test, valid = train_test_split(temp, test_size=0.5, shuffle=True, random_state=0, stratify=temp['CATEGORY'])

# データの保存
! mkdir -p /content/data/
train.to_csv('/content/data/train.txt', sep="\t", index=False)
test.to_csv('/content/data/test.txt', sep="\t", index=False)
valid.to_csv('/content/data/valid.txt', sep="\t", index=False)

# print(train['CATEGORY'].value_counts())
# print(valid['CATEGORY'].value_counts())
# print(test['CATEGORY'].value_counts())

In [3]:
import torch
import numpy as np
import string
def transform_w2v(text):
    # maketransの説明  maketrans(変換前文字列, 変換後文字列, 削除対象文字列)
    # string.punctuationの中身  !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
    table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    words = text.translate(table).split()  
    vec = [model[word] for word in words if word in model] 
    return torch.tensor(sum(vec) / len(vec))

def make_vec(input, features_output, labels_output):
    with open(input, 'r') as f:
        category_dict = {'b': 0, 't': 1, 'e':2, 'm':3}
        data  = pd.read_csv(input, sep='\t')
        sentences = data["TITLE"]
        labels = data["CATEGORY"]

        X = torch.stack([transform_w2v(text) for text in sentences])
        Y = torch.tensor(data["CATEGORY"].map(category_dict))
        torch.save(X, features_output)
        torch.save(Y, labels_output)
        return X, Y
! mkdir -p /content/pytorch_data/
X_train, Y_train = make_vec("/content/data/train.txt", "/content/pytorch_data/X_train.pt", "/content/pytorch_data/Y_train.pt")
X_valid, Y_valid = make_vec("/content/data/valid.txt", "/content/pytorch_data/X_valid.pt", "/content/pytorch_data/Y_valid.pt")
X_test , Y_test  = make_vec("/content/data/test.txt" , "/content/pytorch_data/X_test.pt" , "/content/pytorch_data/Y_test.pt")

In [4]:
# ============
# 71. 単層ニューラルネットワークによる予測
# ============
import torch.nn as nn
import numpy as np

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(300, 4, bias=False)
        nn.init.normal_(self.fc.weight, 0.0, 1.0) # 正規乱数で重みを初期化

    def forward(self, x):
        y = self.fc(x)
        return y

net = Net() # ネットワークのインスタンスを作成
y_hat_1 = torch.softmax(net(X_train[:1]), dim=-1)
print(y_hat_1)
Y_hat = torch.softmax(net(X_train[:4]), dim=-1)
print(Y_hat)

tensor([[0.2455, 0.2214, 0.0554, 0.4777]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2455, 0.2214, 0.0554, 0.4777],
        [0.0285, 0.4506, 0.4651, 0.0559],
        [0.0457, 0.1363, 0.2585, 0.5594],
        [0.0236, 0.3270, 0.2822, 0.3673]], grad_fn=<SoftmaxBackward0>)


In [5]:
# ============
# 72. 損失と勾配の計算
# ============
criterion = nn.CrossEntropyLoss()
l_1 = criterion(net(X_train[:1]), Y_train[:1])  # 事例 x_1 の損失の計算
net.zero_grad() # 勾配を初期化
l_1.backward()  # 勾配を計算
print(f'損失: {l_1:.4f}')
print(f'勾配:\n{net.fc.weight.grad}')

l_1_4 = criterion(net(X_train[:4]), Y_train[:4]) # 事例 x_1 ～ x_4 の損失の計算
net.zero_grad()
l_1_4.backward()
print(f'損失: {l_1_4:.4f}')
print(f'勾配:\n{net.fc.weight.grad}')

# 損失の確認
ans = [] 
for s,i in zip(torch.softmax(net(X_train[:4]), dim=-1),Y_train[:4]):
    s = s.to('cpu').detach().numpy().copy()
    ans.append(-np.log(s[i]))
print(np.mean(ans))

損失: 1.4045
勾配:
tensor([[ 0.1056, -0.0394,  0.0544,  ..., -0.0229, -0.0169,  0.0616],
        [-0.0310,  0.0116, -0.0160,  ...,  0.0067,  0.0050, -0.0181],
        [-0.0078,  0.0029, -0.0040,  ...,  0.0017,  0.0012, -0.0045],
        [-0.0669,  0.0250, -0.0345,  ...,  0.0145,  0.0107, -0.0390]])
損失: 2.5158
勾配:
tensor([[ 0.0652, -0.0227, -0.0069,  ..., -0.0288, -0.0246,  0.0257],
        [-0.0253,  0.0043,  0.0069,  ...,  0.0131,  0.0137, -0.0091],
        [-0.0067,  0.0127, -0.0138,  ...,  0.0122, -0.0100, -0.0018],
        [-0.0332,  0.0057,  0.0138,  ...,  0.0036,  0.0210, -0.0148]])
2.5158274


In [6]:
# ============
# 73. 確率的勾配降下法による学習
# ============
from torch.utils.data import DataLoader, TensorDataset

# ランダムシード
torch.manual_seed(42)

# バッチサイズ
batch_size = 10

# モデルをGPUへ転送
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)

# Datasetの作成
train = TensorDataset(X_train, Y_train)   
valid = TensorDataset(X_valid, Y_valid)
test  = TensorDataset(X_test,  Y_test)

# 訓練用・検証用・評価用の各データをDataLoaderに格納
train_loader = DataLoader(train, batch_size, shuffle=True)
val_loader = DataLoader(valid, batch_size)
test_loader = DataLoader(test, batch_size)

# 損失関数と最適化手法の選択
criterion = nn.CrossEntropyLoss()   
optimizer = torch.optim.SGD(net.parameters(), lr=0.01) 

# 訓練ループ（エポックごとに検証データの正解率を表示）
max_epoch = 30

for epoch in range(max_epoch):
    train_loss = 0
    for x, t in train_loader:

        # データをGPUへ転送
        x = x.to(device)
        t = t.to(device)

        # 勾配を初期化
        optimizer.zero_grad()

        # 順伝播
        y = net(x)   # 2
        loss = criterion(y, t)   # 3

        # 学習状況の確認
        # print("epoch: %d    loss: %.3f" % (epoch+1, loss.item()))
        train_loss += loss

        # 誤差逆伝播
        loss.backward()   # 4
        optimizer.step()  # 5
    
    print("epoch: %d    train_loss: %.3f" % (epoch+1, train_loss/len(train_loader)))

epoch: 1    train_loss: 1.419
epoch: 2    train_loss: 1.035
epoch: 3    train_loss: 0.910
epoch: 4    train_loss: 0.837
epoch: 5    train_loss: 0.783
epoch: 6    train_loss: 0.742
epoch: 7    train_loss: 0.707
epoch: 8    train_loss: 0.678
epoch: 9    train_loss: 0.653
epoch: 10    train_loss: 0.631
epoch: 11    train_loss: 0.612
epoch: 12    train_loss: 0.594
epoch: 13    train_loss: 0.579
epoch: 14    train_loss: 0.565
epoch: 15    train_loss: 0.551
epoch: 16    train_loss: 0.540
epoch: 17    train_loss: 0.529
epoch: 18    train_loss: 0.519
epoch: 19    train_loss: 0.509
epoch: 20    train_loss: 0.501
epoch: 21    train_loss: 0.493
epoch: 22    train_loss: 0.486
epoch: 23    train_loss: 0.478
epoch: 24    train_loss: 0.471
epoch: 25    train_loss: 0.465
epoch: 26    train_loss: 0.459
epoch: 27    train_loss: 0.454
epoch: 28    train_loss: 0.448
epoch: 29    train_loss: 0.443
epoch: 30    train_loss: 0.439


In [7]:
# ============
# 74. 正解率の計算
# ============
# ラベルを格納するリスト↓
labels1 = []
labels2 = []

def calc_acc(data_loader):
    loss = 0
    with torch.no_grad():
        accs = list()
        for x, t in data_loader:
            x = x.to(device)
            t = t.to(device)
            labels2.extend(t)

            y = net(x)
            loss += criterion(y, t).item()

            label = torch.argmax(y, dim=1)
            labels1.extend(label)
            
            acc = (label == t).sum() * 1.0 / len(t)
            accs.append(acc)

    return torch.tensor(accs).mean(), loss/len(data_loader)
    

# 訓練ループ（エポックごとに検証データの正解率を表示）
max_epoch = 30

for epoch in range(max_epoch):
    for x, t in train_loader:

        # データをGPUへ転送
        x = x.to(device)
        t = t.to(device)

        # 勾配を初期化
        optimizer.zero_grad()

        # 順伝播
        y = net(x)   # 2
        loss = criterion(y, t)   # 3

        # 学習状況の確認
        label = torch.argmax(y, dim=1)
        acc = (label == t).sum() * 1.0 / len(t)
        # print("epoch: %d    loss: %.3f    acc: %.3f" % (epoch+1, loss.item(), acc))

        # 誤差逆伝播
        loss.backward()   # 4
        optimizer.step()  # 5
    
    # 検証データの性能を確認
    val_acc, val_loss = calc_acc(val_loader)
    print("epoch: %d    val_acc: %.3f    val_acc: %.3f" % (epoch+1, val_acc, val_loss))


# 評価データで性能確認（推定値と目標値を評価データの先頭10件分表示し、評価データの正解率を表示）
print("出力:", ["%d" % p for p in labels1[:10]])
print("正解:", ["%d" % p for p in labels2[:10]])
train_acc, train_loss = calc_acc(train_loader)
print("学習データの正解率:", train_acc)
test_acc, test_loss = calc_acc(test_loader)
print("評価データの正解率:", test_acc)

epoch: 1    val_acc: 0.831    val_acc: 0.461
epoch: 2    val_acc: 0.831    val_acc: 0.457
epoch: 3    val_acc: 0.833    val_acc: 0.453
epoch: 4    val_acc: 0.832    val_acc: 0.449
epoch: 5    val_acc: 0.835    val_acc: 0.446
epoch: 6    val_acc: 0.835    val_acc: 0.442
epoch: 7    val_acc: 0.839    val_acc: 0.439
epoch: 8    val_acc: 0.840    val_acc: 0.436
epoch: 9    val_acc: 0.840    val_acc: 0.433
epoch: 10    val_acc: 0.841    val_acc: 0.430
epoch: 11    val_acc: 0.841    val_acc: 0.427
epoch: 12    val_acc: 0.844    val_acc: 0.424
epoch: 13    val_acc: 0.844    val_acc: 0.421
epoch: 14    val_acc: 0.845    val_acc: 0.419
epoch: 15    val_acc: 0.845    val_acc: 0.417
epoch: 16    val_acc: 0.846    val_acc: 0.414
epoch: 17    val_acc: 0.846    val_acc: 0.412
epoch: 18    val_acc: 0.846    val_acc: 0.410
epoch: 19    val_acc: 0.846    val_acc: 0.408
epoch: 20    val_acc: 0.848    val_acc: 0.406
epoch: 21    val_acc: 0.849    val_acc: 0.404
epoch: 22    val_acc: 0.849    val_acc: 0.4

In [ ]:
# ============
# 75. 損失と正解率のプロット tensorboardを利用しました。重かったため結果は消しています。
# ============
%load_ext tensorboard
!rm -rf ./runs
%tensorboard --logdir ./runs
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

# ラベルを格納するリスト↓
labels1 = []
labels2 = []

# 訓練ループ（エポックごとに検証データの正解率を表示）
max_epoch = 30

for epoch in range(max_epoch):
    for x, t in train_loader:

        # データをGPUへ転送
        x = x.to(device)
        t = t.to(device)

        # 勾配を初期化
        optimizer.zero_grad()

        # 順伝播
        y = net(x)   # 2
        loss = criterion(y, t)   # 3

        # 学習状況の確認
        label = torch.argmax(y, dim=1)
        acc = (label == t).sum() * 1.0 / len(t)
        # print("epoch: %d    loss: %.3f    acc: %.3f" % (epoch+1, loss.item(), acc))

        # 誤差逆伝播
        loss.backward()   # 4
        optimizer.step()  # 5
    
    # 訓練データの性能を確認
    train_acc, train_loss = calc_acc(train_loader)
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Accuracy/train', train_acc, epoch)
    
    # 検証データの性能を確認
    val_acc, val_loss = calc_acc(val_loader)
    writer.add_scalar('Loss/valid', val_loss, epoch)
    writer.add_scalar('Accuracy/valid', val_acc, epoch)
    print("epoch: %d    val_acc: %.3f    val_loss: %.3f" % (epoch+1, val_acc, val_loss))


# 評価データで性能確認（推定値と目標値を評価データの先頭10件分表示し、評価データの正解率を表示）
print("出力:", ["%d" % p for p in labels1[:10]])
print("正解:", ["%d" % p for p in labels2[:10]])
test_acc, test_loss = calc_acc(test_loader)
print("正解率:", test_acc)

In [8]:
# ============
# 76. チェックポイント
# ============
# 保存するディレクトリの作成
! rm -r /content/checkpoints
! mkdir -p /content/checkpoints

# ラベルを格納するリスト↓
labels1 = []
labels2 = []

# 訓練ループ（エポックごとに検証データの正解率を表示）
max_epoch = 30

for epoch in range(max_epoch):
    for x, t in train_loader:

        # データをGPUへ転送
        x = x.to(device)
        t = t.to(device)

        # 勾配を初期化
        optimizer.zero_grad()

        # 順伝播
        y = net(x)   # 2
        loss = criterion(y, t)   # 3

        # 学習状況の確認
        label = torch.argmax(y, dim=1)
        acc = (label == t).sum() * 1.0 / len(t)
        # print("epoch: %d    loss: %.3f    acc: %.3f" % (epoch+1, loss.item(), acc))

        # 誤差逆伝播
        loss.backward()   # 4
        optimizer.step()  # 5
    
    # 訓練データの性能を確認
    train_acc, train_loss = calc_acc(train_loader)

    # チェックポイントの保存
    torch.save(net.state_dict(), '/content/checkpoints/'+'checkpoint'+str(epoch)+'.model')
    torch.save(optimizer.state_dict(), '/content/checkpoints/'+'checkpoint'+str(epoch)+'.param')

    
    # 検証データの性能を確認
    val_acc, val_loss = calc_acc(val_loader)
    print("epoch: %d    val_acc: %.3f    val_loss: %.3f" % (epoch+1, val_acc, val_loss))
    
# 評価データで性能確認（推定値と目標値を評価データの先頭10件分表示し、評価データの正解率を表示）
print("出力:", ["%d" % p for p in labels1[:10]])
print("正解:", ["%d" % p for p in labels2[:10]])
test_acc, test_loss = calc_acc(test_loader)
print("正解率:", test_acc)

rm: cannot remove '/content/checkpoints': No such file or directory
epoch: 1    val_acc: 0.856    val_loss: 0.387
epoch: 2    val_acc: 0.856    val_loss: 0.385
epoch: 3    val_acc: 0.857    val_loss: 0.384
epoch: 4    val_acc: 0.858    val_loss: 0.382
epoch: 5    val_acc: 0.860    val_loss: 0.381
epoch: 6    val_acc: 0.860    val_loss: 0.380
epoch: 7    val_acc: 0.860    val_loss: 0.378
epoch: 8    val_acc: 0.863    val_loss: 0.377
epoch: 9    val_acc: 0.863    val_loss: 0.376
epoch: 10    val_acc: 0.864    val_loss: 0.375
epoch: 11    val_acc: 0.865    val_loss: 0.373
epoch: 12    val_acc: 0.865    val_loss: 0.372
epoch: 13    val_acc: 0.864    val_loss: 0.371
epoch: 14    val_acc: 0.865    val_loss: 0.370
epoch: 15    val_acc: 0.864    val_loss: 0.369
epoch: 16    val_acc: 0.864    val_loss: 0.368
epoch: 17    val_acc: 0.864    val_loss: 0.367
epoch: 18    val_acc: 0.865    val_loss: 0.366
epoch: 19    val_acc: 0.866    val_loss: 0.365
epoch: 20    val_acc: 0.867    val_loss: 0.364
e

In [11]:
# ============
# 77. ミニバッチ化 
# ============
"""
ミニバッチ化の機能はすでにあるのでバッチサイズのみ変えて学習を行う
"""
# ランダムシード
torch.manual_seed(42)

# バッチサイズ
batch_size = 32

# 損失関数と最適化手法の選択
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

# 訓練用・検証用・評価用の各データをDataLoaderに格納
train_loader = DataLoader(train, batch_size, shuffle=True)
val_loader = DataLoader(valid, batch_size)
test_loader = DataLoader(test, batch_size)

# 保存するディレクトリの作成
# ! mkdir -p /content/checkpoints

# ラベルを格納するリスト↓
labels1 = []
labels2 = []

# 訓練ループ（エポックごとに検証データの正解率を表示）
max_epoch = 30

for epoch in range(max_epoch):
    for x, t in train_loader:

        # データをGPUへ転送
        x = x.to(device)
        t = t.to(device)

        # 勾配を初期化
        optimizer.zero_grad()

        # 順伝播
        y = net(x)   # 2
        loss = criterion(y, t)   # 3

        # 学習状況の確認
        label = torch.argmax(y, dim=1)
        acc = (label == t).sum() * 1.0 / len(t)
        # print("epoch: %d    loss: %.3f    acc: %.3f" % (epoch+1, loss.item(), acc))

        # 誤差逆伝播
        loss.backward()   # 4
        optimizer.step()  # 5
    
    # 訓練データの性能を確認
    train_acc, train_loss = calc_acc(train_loader)

    # チェックポイントの保存
    # torch.save(net.state_dict(), '/content/checkpoints/'+'checkpoint'+str(epoch)+'.model')
    # torch.save(optimizer.state_dict(), '/content/checkpoints/'+'checkpoint'+str(epoch)+'.param')

    
    # 検証データの性能を確認
    val_acc, val_loss = calc_acc(val_loader)
    print("epoch: %d    val_acc: %.3f    val_loss: %.3f" % (epoch+1, val_acc, val_loss))

# 評価データで性能確認（推定値と目標値を評価データの先頭10件分表示し、評価データの正解率を表示）
print("出力:", ["%d" % p for p in labels1[:10]])
print("正解:", ["%d" % p for p in labels2[:10]])
test_acc, test_loss = calc_acc(test_loader)
print("正解率:", test_acc)

epoch: 1    val_acc: 0.901    val_loss: 0.275
epoch: 2    val_acc: 0.900    val_loss: 0.275
epoch: 3    val_acc: 0.900    val_loss: 0.275
epoch: 4    val_acc: 0.900    val_loss: 0.275
epoch: 5    val_acc: 0.900    val_loss: 0.275
epoch: 6    val_acc: 0.900    val_loss: 0.275
epoch: 7    val_acc: 0.900    val_loss: 0.275
epoch: 8    val_acc: 0.901    val_loss: 0.275
epoch: 9    val_acc: 0.900    val_loss: 0.275
epoch: 10    val_acc: 0.900    val_loss: 0.275
epoch: 11    val_acc: 0.899    val_loss: 0.275
epoch: 12    val_acc: 0.900    val_loss: 0.275
epoch: 13    val_acc: 0.900    val_loss: 0.275
epoch: 14    val_acc: 0.899    val_loss: 0.275
epoch: 15    val_acc: 0.900    val_loss: 0.275
epoch: 16    val_acc: 0.900    val_loss: 0.275
epoch: 17    val_acc: 0.902    val_loss: 0.275
epoch: 18    val_acc: 0.900    val_loss: 0.275
epoch: 19    val_acc: 0.899    val_loss: 0.275
epoch: 20    val_acc: 0.901    val_loss: 0.275
epoch: 21    val_acc: 0.901    val_loss: 0.274
epoch: 22    val_acc: 

In [ ]:
# ============
# 78. GPU上での学習
# ============
"""
すでにGPU上で学習をしていたためスキップ
"""

In [21]:
# ============
# 79. 多層ニューラルネットワーク
# ============
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 4)

    def forward(self, x):
        h = F.relu(self.fc1(x))  # xを線形変換（fc1）し、非線形変換（relu）し、中間表現hを得る
        y = self.fc2(h)
        return y

# ネットワークのインスタンスを作成
net = Net() 

# バッチサイズ
batch_size = 32

# 訓練用・検証用・評価用の各データをDataLoaderに格納
train_loader = DataLoader(train, batch_size, shuffle=True)
val_loader = DataLoader(valid, batch_size)
test_loader = DataLoader(test, batch_size)

# 損失関数と最適化手法の選択
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.05)

# モデルをGPUへ転送
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)

# バッチサイズ
batch_size = 64

# 保存するディレクトリの作成
# ! mkdir -p /content/checkpoints

# ラベルを格納するリスト↓
labels1 = []
labels2 = []

def calc_acc(data_loader):
    loss = 0
    with torch.no_grad():
        accs = list()
        for x, t in data_loader:
            x = x.to(device)
            t = t.to(device)
            labels2.extend(t)

            y = net(x)
            loss += criterion(y, t).item()

            label = torch.argmax(y, dim=1)
            labels1.extend(label)
            
            acc = (label == t).sum() * 1.0 / len(t)
            accs.append(acc)

    return torch.tensor(accs).mean(), loss/len(data_loader)

# 訓練ループ（エポックごとに検証データの正解率を表示）
max_epoch = 20

for epoch in range(max_epoch):
    for x, t in train_loader:

        # データをGPUへ転送
        x = x.to(device)
        t = t.to(device)

        # 勾配を初期化
        optimizer.zero_grad()

        # 順伝播
        y = net(x)   # 2
        loss = criterion(y, t)   # 3

        # 学習状況の確認
        label = torch.argmax(y, dim=1)
        acc = (label == t).sum() * 1.0 / len(t)
        # print("epoch: %d    loss: %.3f    acc: %.3f" % (epoch+1, loss.item(), acc))

        # 誤差逆伝播
        loss.backward()   # 4
        optimizer.step()  # 5
    
    # 訓練データの性能を確認
    train_acc, train_loss = calc_acc(train_loader)

    # チェックポイントの保存
    # torch.save(net.state_dict(), '/content/checkpoints/'+'checkpoint'+str(epoch)+'.model')
    # torch.save(optimizer.state_dict(), '/content/checkpoints/'+'checkpoint'+str(epoch)+'.param')

    
    # 検証データの性能を確認
    val_acc, val_loss = calc_acc(val_loader)
    print("epoch: %d    val_acc: %.3f    val_acc: %.3f" % (epoch+1, val_acc, val_loss))

# 評価データで性能確認（推定値と目標値を評価データの先頭10件分表示し、評価データの正解率を表示）
print("出力:", ["%d" % p for p in labels1[:10]])
print("正解:", ["%d" % p for p in labels2[:10]])
test_acc, test_loss = calc_acc(test_loader)
print("正解率:", test_acc)

epoch: 1    val_acc: 0.771    val_acc: 0.780
epoch: 2    val_acc: 0.774    val_acc: 0.594
epoch: 3    val_acc: 0.794    val_acc: 0.526
epoch: 4    val_acc: 0.820    val_acc: 0.470
epoch: 5    val_acc: 0.853    val_acc: 0.420
epoch: 6    val_acc: 0.874    val_acc: 0.379
epoch: 7    val_acc: 0.880    val_acc: 0.349
epoch: 8    val_acc: 0.886    val_acc: 0.334
epoch: 9    val_acc: 0.893    val_acc: 0.317
epoch: 10    val_acc: 0.894    val_acc: 0.302
epoch: 11    val_acc: 0.892    val_acc: 0.297
epoch: 12    val_acc: 0.895    val_acc: 0.291
epoch: 13    val_acc: 0.900    val_acc: 0.283
epoch: 14    val_acc: 0.899    val_acc: 0.288
epoch: 15    val_acc: 0.901    val_acc: 0.277
epoch: 16    val_acc: 0.906    val_acc: 0.274
epoch: 17    val_acc: 0.897    val_acc: 0.276
epoch: 18    val_acc: 0.906    val_acc: 0.272
epoch: 19    val_acc: 0.909    val_acc: 0.269
epoch: 20    val_acc: 0.903    val_acc: 0.275
出力: ['2', '2', '2', '2', '2', '2', '2', '2', '0', '2']
正解: ['2', '2', '2', '2', '3', '0',